In [ ]:
import h5py
import numpy as np
from scipy.interpolate import interp1d
from scipy.stats import pearsonr
from astropy.cosmology import WMAP9 as cosmo

import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patheffects as PathEffects

import profiles
import cluster

import emcee
from multiprocessing import Pool

from colossus.halo import splashback
from colossus.cosmology import cosmology
cosmology.setCosmology('planck18')

cluster_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 21, 22, 24, 25, 28, 29]

In [ ]:

def get_profile(idx, type='dm', r_min=0.2, r_max=20., r_min_dm=0.3, nbins=30, zlabel=''):
    # Get a radial profile as a function of 3d radius. 
    
    
    CEidx = cluster_indices[idx]
    info = np.load('data/sub/hydro'+zlabel+'/'+str(CEidx)+'info.npy', allow_pickle=True)
    
    if(type=='dm'):
        file_data = h5py.File('data/mass_profiles'+zlabel+'.hdf5', 'r')
        x = np.array(file_data['BinLims'])
        y = np.array(file_data['DensityProfile3D'][idx, :, 4])
        
        y = y[(x>r_min_dm) & (x<r_max)]/info[0]/1e10
        x = x[(x>r_min_dm) & (x<r_max)]
        
    else:

        bins = np.geomspace(r_min, r_max, nbins)
        rbin= ((bins[1:]**3. + bins[:-1]**3.) / 2.)**(1./3.)
        volume = 4./3. * np.pi *(bins[1:]**3. - bins[:-1]**3.)        
        
        posc = info[1]
        
        if(type=='sub'):
            mass, pos = np.load('data/sub/hydro'+zlabel+'/'+str(CEidx)+'mass.npy', allow_pickle=True), \
                            np.load('data/sub/hydro'+zlabel+'/'+str(CEidx)+'pos.npy', allow_pickle=True)
        if(type=='gal'):
            mstar, pos = np.load('data/galaxies'+zlabel+'/'+str(CEidx)+'mstar.npy', allow_pickle=True), \
                        np.load('data/galaxies'+zlabel+'/'+str(CEidx)+'pos.npy', allow_pickle=True)


        r = np.sqrt( ( (pos-posc)**2.).sum(axis=1))

        if(type=='sub'):
            r = r[(r>r_min) & (r<r_max) & (np.log10(mass)>-1)]
        if(type=='gal'):
            r = r[(r>r_min) & (r<r_max) & (mstar>8.)]

    
        y = np.histogram(r, bins=bins)[0]/volume/len(r)
        x = rbin

    return x, y 

In [ ]:
def normal(x, c, s):
    return -((x-c)**2.)/2./(s**2.)

def lnlike(params, x, y, errors):
    model = profiles.rho(x, params)
    return -0.5*((y-model)**2./errors**2.).sum()

def lnpost(params, x, y, errors, logalphaprior):
    # Posterior for fit rho, uses lnlike as likelihood
    
    rho_s, r_s, logalpha, r_t, logbeta, loggamma, rho_0, s_e = params
    params = np.array([rho_s, r_s, logalpha, r_t, logbeta, loggamma, rho_0, s_e])
    if( (0.<rho_s<1e10) and (0.<rho_0 < 1e10) and (0.1 <= r_s <= 10.)and (0.1 <= r_t <= 20.) and (0.1<s_e<2.)):
        return lnlike(params, x, y, errors) + \
                normal(logalpha, np.log10(0.20), logalphaprior) +\
                normal(logbeta, np.log10(6.), 0.2) +\
                normal(loggamma, np.log10(4.), 0.2) +\
                normal(r_t, 4., 2.)
    return -np.inf

pool = Pool(2)

def fit_rho(x, y, errors, p0=np.array([ 1e-2,  0.5,  -0.3,  1.,  0.6,  0.8, 1e-4,  1.5]), \
            logalphaprior=0.2, \
            nsteps=2500):
    # Fit a 3d profile rho(r) with a DK14 model. 
    
    nwalkers = 32
    ndim = 8

    pos = [p0 + p0*np.random.randn(ndim)*1e-4 for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnpost, args=(x, y, errors, logalphaprior), pool=pool)
    sampler.run_mcmc(pos, nsteps=nsteps, progress=True)

    lnprob = sampler.get_log_prob()
    params = sampler.get_chain(flat=True)[lnprob.argmax()]

    return sampler.get_chain(flat=True), lnprob[:, :].max(axis=1), params

TABLE AND LOAD PARAMETERS
-------------------------

In [ ]:
M200ms = [None]*len(cluster_indices)
M_sps = [None]*len(cluster_indices)
M_sps_gal = [None]*len(cluster_indices)
M_sps_sub = [None]*len(cluster_indices)
Gdyns = [None]*len(cluster_indices)
r_sps = [None]*len(cluster_indices)
r_sps_sub = [None]*len(cluster_indices)
r_sps_gal = [None]*len(cluster_indices)
R200ms = [None]*len(cluster_indices)
s_es = [None]*len(cluster_indices)
s_es_gal = [None]*len(cluster_indices)
s_es_sub = [None]*len(cluster_indices)

for idx, CEidx in enumerate(cluster_indices):
    H = cluster.cluster(idx, zlabel='')
    M200ms[idx] = H.M200m
    Gdyns[idx] = H.Gdyn
    r_sps_sub[idx] = H.r_sp_sub
    r_sps_gal[idx] = H.r_sp_gal
    M_sps_gal[idx] = H.M_sp_gal
    M_sps_sub[idx] = H.M_sp_sub
    R200ms[idx] = H.R200m
    s_es[idx] = H.dmbfparams[-1]
    s_es_gal[idx] = H.galbfparams[-1]
    s_es_sub[idx] = H.subbfparams[-1]
    r_sps[idx] = H.r_sp
    M_sps[idx] = H.M_sp
    
M200ms = 10**np.array(M200ms)
M_sps =  np.array(M_sps)
M_sps_gal =  np.array(M_sps_gal)
M_sps_sub =  np.array(M_sps_sub)
Gdyns =  np.array(Gdyns)
r_sps =  np.array(r_sps)
r_sps_sub =  np.array(r_sps_sub)
r_sps_gal =  np.array(r_sps_gal)
R200ms =  np.array(R200ms)
s_es =  np.array(s_es)
s_es_gal =  np.array(s_es_gal)
s_es_sub =  np.array(s_es_sub)


for idx, CEidx in enumerate(cluster_indices):
    print("CE-%s & %.1f & %.2f & %.2f & %.2f & %.2f & %.2f \\\\"%(CEidx, Gdyns[idx], M200ms[idx]/1e14, R200ms[idx], r_sps[idx], r_sps_gal[idx], r_sps_sub[idx]) )



FIGURE 1: Stacked subhalo profiles DMO vs HYDRO
----------

In [ ]:
rbins = np.linspace(0.2, 5.)
p0 = np.array([ 71.7801364 ,   7.11055532,  -0.60918457,  10.6156843 ,
          0.22011404,   2.15387829, 107.06723642,   1.12129605])
ytot = np.zeros(len(rbins)-1)
for idx, CEidx in enumerate(cluster_indices):

    H = cluster.cluster(idx)
    
    x, y = H.get_profile(rbins*H.R200m, sub=True)
    x = x/H.R200m
    ytot += y
    
samples, lnprob, bfparams = fit_rho(x, ytot, errors=ytot/1000., nsteps=50, p0=p0)
plt.loglog(x, ytot)
plt.loglog(x, profiles.rho(x, bfparams))

p0 = np.array([ 3.08164902e+02,  2.69225938e+00, -5.04048305e-01,  4.70352562e+00,
         2.43272615e-01,  1.60947206e+00,  1.37914752e+02,  1.11461176e+00])
ytot2 = np.zeros(len(rbins)-1)
for idx, CEidx in enumerate(cluster_indices):

    H = cluster.cluster(idx)
    
    x, y = H.get_profile(rbins*H.R200m, sub=True, label='dm')
    x = x/H.R200m
    ytot2 += y

samples2, lnprob, bfparams2 = fit_rho(x, ytot2, errors=ytot2/1000., nsteps=50, p0=p0)
plt.loglog(x, ytot2)
plt.loglog(x, profiles.rho(x, bfparams2))
plt.show()

In [ ]:
plt.figure(figsize=(5,4))
rplot = np.geomspace(0.1, 10, 100)
logy_tot = np.log(profiles.rho(rplot, bfparams))
derivative = np.gradient(logy_tot, np.log(rplot))

plt.semilogx(rplot, derivative, c='steelblue', label='Hydro', lw=2)


logy_tot = np.log(profiles.rho(rplot, bfparams2))
derivative2 = np.gradient(logy_tot, np.log(rplot))

plt.gca().set_xlabel(r'$r/r_\mathrm{200m}$')
plt.gca().set_ylabel(r'$d\,\log n_s(r)/d\,\log r$')
plt.semilogx(rplot, derivative2, ls='--', c='coral', label='DMO', lw=2)

plt.gca().axes.xaxis.set_ticks([0.1, 1, 10])
plt.gca().axes.xaxis.set_ticklabels([0.1, 1, 10])

ia = plt.gca().inset_axes([0.2, 0.5, 0.3, 0.2])

ia.semilogx(rplot, derivative/derivative2, c='k', lw=0.5)
ia.set_xticks([])
ia.set_yticks([0.99, 1, 1.01])
ia.set_title('ratio')

plt.tight_layout()
plt.legend(frameon=False, loc=4)
plt.savefig('figs/1comparison.pdf')
plt.show()

FIGURE 3: FIT EXAMPLES
------------------------

In [ ]:

fig = plt.figure(figsize=(5, 6))
gs = gridspec.GridSpec(5, 2)


axtitle = fig.add_subplot(gs[:, :])
axtitle.set_xticks([])
axtitle.set_yticks([])

axtitle.set_xlabel('\n\n$r$ (Mpc)')
axtitle.spines["right"].set_visible(False)
axtitle.spines["left"].set_visible(False)
axtitle.spines["bottom"].set_visible(False)
axtitle.spines["top"].set_visible(False)


ax1 = fig.add_subplot(gs[0:3, 0])
ax2 = fig.add_subplot(gs[0:3, 1])
ax3 = fig.add_subplot(gs[3:, 0])
ax4 = fig.add_subplot(gs[3:, 1])




rmax = 12.



for idx, CEidx in enumerate(cluster_indices):
    #print(idx)
    for type in ['dm', 'gal','sub']:
        if(type!='sub'):
            continue
        if(idx!=16) and (idx!=9):
            continue
        
       
        H = cluster.cluster(idx)
        x, y = get_profile(idx, type=type)
        
        H = cluster.cluster(idx)
        
        if(idx==9):
            
            H = cluster.cluster(idx)
            norm = y.sum()
            y = y/norm
            print(norm)
            
            
            img, r, vr = H.get_ps(psrbins= np.linspace(np.log10(1), np.log10(rmax), 50), psvbins= np.linspace(-2.5e3, 2.5e3, 50))
            #ax3.scatter(np.log10(r[r<rmax]), vr[r<rmax], s=0.01, c='coral')
            img[img==0] = np.nan
            ax3.imshow(img[::, ::-1].T, extent=[0., np.log10(rmax), -2.5e3,2.5e3], aspect='auto', vmax=5e-4, cmap='Reds')
            ax3.axvline(np.log10(H.r_sp_sub), c='k', lw=0.5)
            ax1.semilogy(np.log10(x[(x<rmax) & (x>1)]), y[(x<rmax) & (x>1)], c='coral')
            
            xplot = np.linspace(1, 20, 100)
            ax1.semilogy(np.log10(xplot), profiles.rho(xplot, H.subbfparams)/norm, c='k', lw=0.5)
            
        else:
            norm = y.sum()
            y = y/norm
            print(norm)
            H = cluster.cluster(idx)
            img, r, vr = H.get_ps(psrbins= np.linspace(np.log10(1), np.log10(rmax), 50), psvbins= np.linspace(-2.5e3, 2.5e3
, 50))
            #ax4.scatter(np.log10(r[r<rmax]), vr[r<rmax], s=0.01, c='steelblue')
            img[img==0] = np.nan
            ax4.imshow(img[::, ::-1].T, extent=[0., np.log10(rmax), -2.5e3,2.5e3], aspect='auto', vmax=5e-4, cmap='Blues')
            ax4.axvline(np.log10(H.r_sp_sub), c='k', lw=0.5)
            ax2.semilogy(np.log10(x[(x<rmax) & (x>1)]), y[(x<rmax) & (x>1)], c='steelblue')

            xplot = np.linspace(1, 20, 100)
            ax2.semilogy(np.log10(xplot), profiles.rho(xplot, H.subbfparams)/norm, c='k', lw=0.5)

ax1.set_xlim([0, np.log10(rmax)])
ax2.set_xlim([0., np.log10(rmax)])
ax3.set_xlim([0, np.log10(rmax)])
ax4.set_xlim([0., np.log10(rmax)])

ax1.set_ylim([2e-3, 5e-1])
ax2.set_ylim([2e-3, 5e-1])
ax3.set_ylim([-2.5e3, 2.5e3])
ax4.set_ylim([-2.5e3, 2.5e3])


ax3.set_xlabel(r"$r$ (Mpc)")
ax4.set_xlabel(r"$r$ (Mpc)")

ax3.set_ylabel(r"$v_r$ (km/s)")

ax1.set_ylabel(r"$n_s(r)/\overline{n}_s$")


ax1.axes.xaxis.set_ticklabels([])
ax2.axes.xaxis.set_ticklabels([])
ax2.axes.yaxis.set_ticklabels([])
ax4.axes.yaxis.set_ticklabels([])


ax1.axes.xaxis.set_ticks([0, 0.6989, 1])
ax2.axes.xaxis.set_ticks([0, 0.6989, 1])
ax3.axes.xaxis.set_ticks([0, 0.6989, 1])
ax4.axes.xaxis.set_ticks([0, 0.6989, 1])
ax3.axes.xaxis.set_ticklabels([1, 5, 10])
ax4.axes.xaxis.set_ticklabels([1, 5, 10])

plt.tight_layout()
plt.savefig("figs/3fitexample.pdf")
plt.show()
            

FIGURE 4: ACCRETION RATE-RSP
-----------------

In [ ]:
plt.figure(figsize=(5, 4))
xplot = np.linspace(0., 4.5, 100)
# Hardcoded fit from paper
plt.plot(xplot, 0.54*(1+0.53*0.27)*(1+1.36*np.exp(-xplot/3)), ls='--', c='darkred', label="More et al. 2015", lw=2)


xdk, ydk = np.loadtxt("DK.txt").T
plt.plot(np.sort(xdk[xdk<5]), np.sort(ydk[xdk<5])[::-1], c='coral', label='Diemer et al. 2017', ls=':', lw=2)

RspR200 = splashback.modelDiemer20RspR200m(xplot, 3, 0, "percentile75")
plt.plot(xplot, RspR200, label='Diemer 2020', c='red', lw=1)


plt.scatter(Gdyns, r_sps/R200ms, c='k', s=20, marker='x')
plt.gca().set_ylim([0.6, 1.8])
plt.gca().set_ylabel(r'$r_\mathrm{sp}/r_{200\mathrm{m}}$')
plt.gca().set_xlabel(r'$\Gamma_{0.3}$')
plt.legend(frameon=False)
plt.tight_layout()
plt.savefig("figs/3gammacorr.pdf")
plt.show()

FIGURE 5: ACCRETION REGION - ACCRETION RATE
---------

In [ ]:

plt.figure(figsize=(5, 4))
plt.scatter(Gdyns, -s_es_sub, c='darkred', marker='x', label='subhaloes', s=20)
plt.scatter(Gdyns, -s_es_gal, c='coral', marker='x', label='galaxies', s=20)
plt.scatter(np.mean(Gdyns), -1.12, marker='*', s=200, label='mean profile', fc='k')

xplot  = np.linspace(-0.5, 5., 100)
plt.fill_between(xplot, -1.15, -1.35, label='Self-similar', facecolor='steelblue', edgecolor=None, alpha=0.2)
plt.gca().set_ylim([-2., 0])
plt.gca().set_xlim([-0.1, 4.2])
plt.gca().set_xlabel("$\Gamma_{0.3}$")
plt.gca().set_ylabel("power-law index")
plt.legend(frameon=False, ncol=2, loc=3)
plt.tight_layout()
plt.savefig("figs/4power.pdf")

print(pearsonr(Gdyns, s_es_sub), pearsonr(Gdyns, s_es_gal))



FIGURE 6: FILAMENTS AND GALAXIES
---------------------


First obtain stacked 2d histograms of the stellar and subhalo populations rotated by an angle defined using the "filaments" and "stars" definition of direction (see analysis.ipynb).

Second, get two density profiles by adding up quadrants in the direction parallel or perpendicular to this angle. 

Third, fit these profiles and obtain best fit parameters.

Fourth and final, plot everything. 

In [ ]:
def normal(x, c, s):
    return -((x-c)**2.)/2./(s**2.)

def lnlike(params, x, y, errors):
    model = profiles.DK14_S(x, params)
    return -0.5*((y-model)**2./errors**2.).sum()

def lnpost(params, x, y, errors, logalphaprior):
    # Posterior for fit_sigma
    rho_s, r_s, logalpha, r_t, logbeta, loggamma, rho_0, s_e = params
    params = np.array([rho_s, r_s, logalpha, r_t, logbeta, loggamma, rho_0, s_e])
    if( (0.<rho_s<1e10) and (0.<rho_0 < 1e10) and (0. <= r_s <= 10) and (0. <= r_t <= 20.) and (0.<s_e<3.)):
        return lnlike(params, x, y, errors) + \
                normal(logalpha, np.log10(0.20), logalphaprior) +\
                normal(logbeta, np.log10(6.), 0.2) +\
                normal(loggamma, np.log10(4.), 0.2) +\
                normal(r_t, 4., 2.)
    return -np.inf

pool = Pool(10)

def fit_sigma(x, y, p0= np.array([1e-2,0.7,0.3, 1.16, 0.4, 0.7, 1e-3, 1.2]), \
            logalphaprior=0.2, \
            nsteps=1000):
    # Fit a projected DK14 profile to a surface density profile as a function of projected radius R
    errors = y/100.
    nwalkers = 16
    ndim = 8

    pos = [p0 + p0*np.random.randn(ndim)*1e-4 for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnpost, args=(x, y, errors, logalphaprior), pool=pool)
    sampler.run_mcmc(pos, nsteps=nsteps, progress=True)

    lnprob = sampler.get_log_prob()
    params = sampler.get_chain(flat=True)[lnprob.argmax()]

    return sampler.get_chain(flat=True), lnprob[:, :].max(axis=1), params

In [ ]:
def rotate(zlabel, t):
    # Load the distribution of subhaloes for each cluster, rotate them using the angle theta associated to the label 
    # t and stack the distributions. 
    
    bins = np.geomspace(0.2, 4, 35) # 0.1, 3, 30
    Rbins = (bins[:-1]+bins[1:])/2
    surface = np.pi*(bins[1:]**2. - bins[:-1]**2.)
    
    imgs = [None]*len(cluster_indices)
    Sigma_major = np.zeros(len(Rbins))
    Sigma_minor = np.zeros(len(Rbins))
    
    for idx, CEidx in enumerate(cluster_indices):
        H = cluster.cluster(idx, zlabel=zlabel)
        r200 = H.R200m
        for dir_idx in [0, 1, 2]:
            if(dir_idx == 0):
                imgs[idx] = H.get_rotated_dist2d(dir_idx, type=t)[0]/3./24.
            else:
                imgs[idx] += H.get_rotated_dist2d(dir_idx, type=t)[0]/3./24.

            y, x = H.get_rotation(dir_idx, type=t)
            x, y = x/r200, y/r200
            R = np.sqrt(y**2.+x**2.)
            Sigma_minor += np.histogram(R[~(abs(y)<abs(x))], bins=bins)[0]/surface
            Sigma_major += np.histogram(R[abs(y)<abs(x)], bins=bins)[0]/surface

    imgs = np.array(imgs)
    img = np.sum(imgs, axis=0)

    Sigma_major = Sigma_major/Sigma_major.sum()
    Sigma_minor = Sigma_minor/Sigma_minor.sum()
    return img, Rbins, Sigma_major, Sigma_minor




img_filaments0, Rbins, Sigma_filament0, Sigma_void0 = rotate('', 'filaments')
img_filaments5, Rbins, Sigma_filament5, Sigma_void5 = rotate('0.5', 'filaments')
img_filaments1, Rbins, Sigma_filament1, Sigma_void1 = rotate('1', 'filaments')

img_stars0, Rbins, Sigma_major0, Sigma_minor0 = rotate('', 'stars')
img_stars5, Rbins, Sigma_major5, Sigma_minor5 = rotate('0.5', 'stars')
img_stars1, Rbins, Sigma_major1, Sigma_minor1 = rotate('1', 'stars')

In [ ]:
# Fit everything!

samples, lnprob, bfparams_filament0 = fit_sigma(Rbins, Sigma_filament0)
samples, lnprob, bfparams_filament5 = fit_sigma(Rbins, Sigma_filament5)
samples, lnprob, bfparams_filament1 = fit_sigma(Rbins, Sigma_filament1)

samples, lnprob, bfparams_void0 = fit_sigma(Rbins, Sigma_void0)
samples, lnprob, bfparams_void5 = fit_sigma(Rbins, Sigma_void5)
samples, lnprob, bfparams_void1 = fit_sigma(Rbins, Sigma_void1)

samples, lnprob, bfparams_major0 = fit_sigma(Rbins, Sigma_major0)
samples, lnprob, bfparams_major5 = fit_sigma(Rbins, Sigma_major5)
samples, lnprob, bfparams_major1 = fit_sigma(Rbins, Sigma_major1)

samples, lnprob, bfparams_minor5 = fit_sigma(Rbins, Sigma_minor5)
samples, lnprob, bfparams_minor1 = fit_sigma(Rbins, Sigma_minor1)
samples, lnprob, bfparams_minor0 = fit_sigma(Rbins, Sigma_minor0)


In [ ]:
fig = plt.figure(figsize=(5, 6))
gs = gridspec.GridSpec(6, 4)

ax1 = fig.add_subplot(gs[:2, :2])

ax2 = fig.add_subplot(gs[:2, 2:])

axtitle = fig.add_subplot(gs[2:, :])
axtitle.set_xticks([])
axtitle.set_yticks([])

ax3 = fig.add_subplot(gs[2:4, :])

ax4 = fig.add_subplot(gs[4:, :])



ax2.imshow(img_stars0, aspect='auto', extent=[-5, 5, -5, 5], cmap='Blues',  vmax=2)
ax1.imshow(img_filaments0, aspect='auto', extent=[-5, 5, -5, 5], cmap='Reds',  vmax=2)
xplot = np.linspace(-5, 5, 100)
ax2.plot(xplot, xplot, c='k', alpha=0.2, ls='-')
ax2.plot(xplot, -xplot, c='k', alpha=0.2, ls='-')
ax1.plot(xplot, xplot, c='k', alpha=0.2, ls='-')
ax1.plot(xplot, -xplot, c='k', alpha=0.2, ls='-')

ax1.errorbar(-3.5, 4.25, xerr=0.5, c='k', capsize=2, capthick=1, elinewidth=1, markeredgewidth=2)
ax1.text(-4.5, 3.1, s="$1~r_\mathrm{200m}$")
ax1.arrow(0, 0, 0, 2,length_includes_head=False,
          head_width=0.4, head_length=0.4, fc='w', color='w')
ax1.arrow(0, 0, 0, -2,length_includes_head=False,
          head_width=0.4, head_length=0.4, fc='w', color='w')
txt = ax1.text(1, -0.5, s="filament", color='k')
txt.set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])


ax2.errorbar(-3.5, 4.25, xerr=0.5, c='k', capsize=2, capthick=1, elinewidth=1, markeredgewidth=2)
ax2.text(-4.5, 3.1, s="$1~r_\mathrm{200m}$")


ax2.arrow(0, 0, 0, 2,length_includes_head=False,
          head_width=0.4, head_length=0.4, fc='w', color='w')
ax2.arrow(0, 0, 0, -2,length_includes_head=False,
          head_width=0.4, head_length=0.4, fc='w', color='w')
txt = ax2.text(2.5, -1.2, s="central \n galaxy \n major axis", color='k', ha='center')
txt.set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])



ax1.set_xticks([])
ax2.set_xticks([])
ax1.set_yticks([])
ax2.set_yticks([])



for bfp, ls, col, lw, ax, label in zip([bfparams_void0, bfparams_filament0,bfparams_minor0, bfparams_major0], 
                        ['-', '-', '-', '-'], 
                        ['darkred', 'coral', 'steelblue', 'skyblue'], 
                        [2, 2, 2, 2],
                        [ax3, ax3, ax4, ax4],
                        ['void', 'filament', 'minor axis', 'major axis']):

    rplot = np.geomspace(0.2, 5., 1000)
    logy_tot = np.log(profiles.rho(rplot, bfp))
    derivative = np.gradient(logy_tot, np.log(rplot))
    ax.semilogx(rplot, derivative, ls=ls, c=col, lw=lw, label=label)    
    
    


ax3.legend(frameon=False)
ax4.legend(frameon=False)
ax4.axes.xaxis.set_ticks([0.2, 1, 5])
ax4.axes.xaxis.set_ticklabels([0.2, 1, 5])
ax3.axes.xaxis.set_ticks([0.2, 1, 5])
ax3.axes.xaxis.set_ticklabels([0.2, 1, 5])
ax4.set_xlabel(r'$r/r_\mathrm{200m}$')
axtitle.set_ylabel('$d\,\log n_s(r)/d\,\logr$\n\n')
axtitle. spines["right"].set_visible(False)
axtitle. spines["left"].set_visible(False)



ax3.set_xticks([])
plt.savefig("figs/6filaments.pdf")
plt.show()


FIGURE 7: MASS-SIZE RELATION
------------------

In [ ]:
def lnpost(params, xs, ys, Gs, noG):
    a, b, a2 = params
    total = 0
    for x, y, G in zip(xs, ys, Gs):
        model = x*3+b + 1.5*np.log10((1.+G))
        if(noG):
            model = x*3+b
        total += normal(y, model, y*0.1)
    return total

def fit_relation(xs, ys, Gs, idxs, noG=False, p0=[3, 13.5, 1.]):
    # Fit the mass size relation using emcee 
    xs, ys = np.array(xs)[idxs], np.array(ys)[idxs]
    nwalkers = 8
    pos = [p0 + p0*np.random.randn(3)*1e-4 for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, 3, lnpost, args=(xs, ys, Gs, noG))
    sampler.run_mcmc(pos, nsteps=1000, progress=True)

    lnprob = sampler.get_log_prob()
    return sampler.get_chain(flat=True)[lnprob.argmax()]

In [ ]:
aci = np.arange(len(cluster_indices))
idxs = aci<100


params = fit_relation(np.log10(r_sps_gal), np.log10(M_sps_gal), Gdyns, idxs=idxs)
params2 = fit_relation(np.log10(r_sps_gal), np.log10(M_sps_gal), Gdyns, noG=True, idxs=idxs)

print(params, params2)

In [ ]:
fig = plt.figure(figsize=(5, 6))

plt.tight_layout()
gs = gridspec.GridSpec(6, 40)

ax1 = fig.add_subplot(gs[:3, 1:])

ax2 = fig.add_subplot(gs[4:, 1:35])


aci = np.arange(len(cluster_indices))
idxs = aci<100

s = ax1.scatter(r_sps_gal,M_sps_gal, c=np.array(Gdyns)[idxs], cmap='binary', marker='x', s=20)

ax1.set_xscale('log')
ax1.set_yscale('log')
plt.colorbar(s, ax=ax1, label=r'$\Gamma_{0.3}$')#, orientation='horizontal')



# Plot median relation
xplot=np.linspace(0.2, .8, 100)
yplot = xplot*params2[0]+params2[1]
ax1.plot(10**xplot, 10**yplot, c='coral', label=r'$\beta=0$')

# Plot median relation with symmetron 
size_corr = np.array([0.1       , 0.05      , 0.03333333, 0.025     , 0.02      ])
r_corr = np.array([0.10945787, 0.13380298, 0.13709141, 0.13698073, 0.13596398])


size_corr_interp = interp1d(size_corr*3/0.05, r_corr, bounds_error=False)(10**xplot)
xplot2 = 10**xplot*(1+size_corr_interp)
ax1.plot(xplot2, 10**yplot, label='Symmetron\n gravity', ls='--', c='steelblue')

ax1.set_xlabel(r'$r_\mathrm{sp}^\mathrm{g}$ (Mpc)')
ax1.set_ylabel(r'$M_\mathrm{sp}^\mathrm{g}$ ($M_\odot$)')
ax1.set_xticks([2, 3, 4, 6])
ax1.axes.xaxis.set_ticklabels([2, 3, 4, 6])
ax1.legend(frameon=False)



ydiff1 = np.log10(M_sps_gal)-np.log10(r_sps_gal)*params[0]-params[1]-params[2]*np.log10((1.+np.array(Gdyns)))
print(np.std(ydiff1[idxs]), " std with gamma")

ydiff2 = np.log10(M_sps_gal)-np.log10(r_sps_gal)*params2[0]-params2[1]
print(np.std(ydiff2[idxs]), " std without gamma")


ax2.scatter(np.array(Gdyns)[idxs],ydiff1[idxs], marker='x', c='darkred', s=20,  label=r'$1.5$')
ax2.scatter(np.array(Gdyns)[idxs],ydiff2[idxs], marker='x', c='coral', s=20, label=r"$0$")
ax2.set_xlabel('$\Gamma_{0.3}$')
ax2.set_ylabel('residuals (dex)')
ax2.axhline(0, c='k', lw=0.2)
plt.gca().set_ylim([-0.7, 0.5])
plt.gca().set_xlim([-0.2, 4.5])
plt.legend(bbox_to_anchor=(1.27, 0.5), frameon=False, ncol=1, loc=7, title=r'$\beta$')

plt.tight_layout()
plt.savefig("figs/7scaling.pdf")
plt.show()

FIGURE 8: REDSHIFT DEPENDENCE OF Gamma-RSP
-----------------------

In [ ]:
M200ms5 = [None]*len(cluster_indices)
M_sps5 = [None]*len(cluster_indices)
M_sps_gal5 = [None]*len(cluster_indices)
M_sps_sub5 = [None]*len(cluster_indices)
Gdyns5 = [None]*len(cluster_indices)
r_sps5 = [None]*len(cluster_indices)
r_sps_sub5 = [None]*len(cluster_indices)
r_sps_gal5 = [None]*len(cluster_indices)
R200ms5 = [None]*len(cluster_indices)
s_es5 = [None]*len(cluster_indices)
s_es_gal5 = [None]*len(cluster_indices)
s_es_sub5 = [None]*len(cluster_indices)

M200ms1 = [None]*len(cluster_indices)
M_sps1 = [None]*len(cluster_indices)
M_sps_gal1 = [None]*len(cluster_indices)
M_sps_sub1 = [None]*len(cluster_indices)
Gdyns1 = [None]*len(cluster_indices)
r_sps1 = [None]*len(cluster_indices)
r_sps_sub1 = [None]*len(cluster_indices)
r_sps_gal1 = [None]*len(cluster_indices)
R200ms1 = [None]*len(cluster_indices)
s_es1 = [None]*len(cluster_indices)
s_es_gal1 = [None]*len(cluster_indices)
s_es_sub1 = [None]*len(cluster_indices)



for idx, CEidx in enumerate(cluster_indices):
    H = cluster.cluster(idx, zlabel='0.5')
    M200ms5[idx] = H.M200m
    Gdyns5[idx] = H.Gdyn
    r_sps_sub5[idx] = H.r_sp_sub
    r_sps_gal5[idx] = H.r_sp_gal
    M_sps_gal5[idx] = H.M_sp_gal
    M_sps_sub5[idx] = H.M_sp_sub
    R200ms5[idx] = H.R200m
    s_es5[idx] = H.dmbfparams[-1]
    s_es_gal5[idx] = H.galbfparams[-1]
    s_es_sub5[idx] = H.subbfparams[-1]
    r_sps5[idx] = H.r_sp
    M_sps5[idx] = H.M_sp
    
    H = cluster.cluster(idx, zlabel='1')
    M200ms1[idx] = H.M200m
    Gdyns1[idx] = H.Gdyn
    r_sps_sub1[idx] = H.r_sp_sub
    r_sps_gal1[idx] = H.r_sp_gal
    M_sps_gal1[idx] = H.M_sp_gal
    M_sps_sub1[idx] = H.M_sp_sub
    R200ms1[idx] = H.R200m
    s_es1[idx] = H.dmbfparams[-1]
    s_es_gal1[idx] = H.galbfparams[-1]
    s_es_sub1[idx] = H.subbfparams[-1]
    r_sps1[idx] = H.r_sp
    M_sps1[idx] = H.M_sp
    
M200ms1 = 1e10*np.array(M200ms1)
M_sps1 =  np.array(M_sps1)
M_sps_gal1 =  np.array(M_sps_gal1)
M_sps_sub1 =  np.array(M_sps_sub1)
Gdyns1 =  np.array(Gdyns1)
r_sps1 =  np.array(r_sps1)
r_sps_sub1 =  np.array(r_sps_sub1)
r_sps_gal1 =  np.array(r_sps_gal1)
R200ms1 =  np.array(R200ms1)
s_es1 =  np.array(s_es1)
s_es_gal1 =  np.array(s_es_gal1)
s_es_sub1 =  np.array(s_es_sub1)

M200ms5 = 1e10*np.array(M200ms5)
M_sps5 =  np.array(M_sps5)
M_sps_gal5 =  np.array(M_sps_gal5)
M_sps_sub5 =  np.array(M_sps_sub5)
Gdyns5 =  np.array(Gdyns5)
r_sps5 =  np.array(r_sps5)
r_sps_sub5 =  np.array(r_sps_sub5)
r_sps_gal5 =  np.array(r_sps_gal5)
R200ms5 =  np.array(R200ms5)
s_es5 =  np.array(s_es5)
s_es_gal5 =  np.array(s_es_gal5)
s_es_sub5 =  np.array(s_es_sub5)

In [ ]:
plt.figure(figsize=(5, 4))
xplot = np.linspace(0., 6, 100)
plt.plot(xplot, 0.54*(1+0.53*0.27)*(1+1.36*np.exp(-xplot/3)), ls='--', c='darkred', label="More et al. 2015", lw=2)


xdk, ydk = np.loadtxt("DK.txt").T
plt.plot(np.sort(xdk[xdk<5]), np.sort(ydk[xdk<5])[::-1], c='coral', label='Diemer et al. 2017', ls=':', lw=2)



s1 = plt.scatter(Gdyns5, r_sps5/R200ms5, c='coral', s=20, marker='x', label='$z=0.5$')
s2 = plt.scatter(Gdyns1, r_sps1/R200ms1, c='skyblue', s=40, marker='+', label='$z=1$')




RspR200 = splashback.modelDiemer20RspR200m(xplot, 3, 0.5, "percentile75")
l1, = plt.plot(xplot, RspR200, c='darkred', lw=1, label='$z=0.5$', ls='--')

RspR200 = splashback.modelDiemer20RspR200m(xplot, 2.5, 1, "percentile75")
l2, = plt.plot(xplot, RspR200, c='steelblue', lw=2, label='$z=1$', ls=':')

RspR200 = splashback.modelDiemer20RspR200m(xplot, 3.5, 0, "percentile75")
l0, = plt.plot(xplot, RspR200, c='k', lw=1, label='$z=0$', ls='-')

legend = plt.gca().legend(handles=[s1, s2], loc=(0.27, 0.075), title='Hydrangea', frameon=False)
plt.gca().add_artist(legend)
legend = plt.gca().legend(handles=[l1, l2, l0], loc=(0.02, 0.01), title='Diemer 2020', frameon=False)
plt.gca().add_artist(legend)


plt.gca().set_ylim([0.2, 1.9])
plt.gca().set_ylabel(r'$r_\mathrm{sp}/r_{200\mathrm{m}}$')
plt.gca().set_xlabel(r'$\Gamma$')
plt.legend(frameon=False, handles=[], loc='upper left')
plt.tight_layout()
plt.savefig("figs/8gammacorrz.pdf")
plt.show()



mod = splashback.modelDiemer20RspR200m(Gdyns, 3, 0, "percentile75")
mod5 = splashback.modelDiemer20RspR200m(Gdyns5, 3, 0.5, "percentile75")
mod1 = splashback.modelDiemer20RspR200m(Gdyns1, 2.5, 1, "percentile75")


#for model, data in zip([mod, mod5, mod1], [r_sps/R200ms, r_sps5/R200ms5, r_sps1/R200ms1]):
pearsonr(Gdyns, r_sps/R200ms), pearsonr(Gdyns5, r_sps5/R200ms5), pearsonr(Gdyns1, r_sps1/R200ms1)

FIGURE 9: FILAMENTS PROFILES AS A FUNCTION OF z
-------------------

In [ ]:
fig = plt.figure(figsize=(5, 4))
gs = gridspec.GridSpec(1, 2)

axtitle = fig.add_subplot(gs[:, :])
axtitle.set_xticks([])
axtitle.set_yticks([])

axtitle.set_xlabel('\n$R/r_\mathrm{200m}$')
axtitle.spines["right"].set_visible(False)
axtitle.spines["left"].set_visible(False)
axtitle.spines["bottom"].set_visible(False)
axtitle.spines["top"].set_visible(False)

ax2 = fig.add_subplot(gs[:, :1])

ax1 = fig.add_subplot(gs[:, 1:])

l1, = ax1.loglog(Rbins, Sigma_major0, c='skyblue', label='0')
ax1.loglog(Rbins, Sigma_minor0, c='skyblue', ls='--')


l2, = ax1.loglog(Rbins, Sigma_major5, c='steelblue', label='0.5')
ax1.loglog(Rbins, Sigma_minor5, c='steelblue', ls='--')

l3, = ax1.loglog(Rbins, Sigma_major1, c='darkblue', label='1')
ax1.loglog(Rbins, Sigma_minor1, c='darkblue', ls='--')

legend = ax1.legend(handles=[l1, l2, l3], loc='upper right', title='z', frameon=False)
ax1.add_artist(legend)
l1, = ax1.plot([1, 1], [100, 100], c='k', label='major axis')
l2, = ax1.plot([1, 1], [100, 100], c='k', ls='--', label='minor axis')
ax1.legend(handles=[l1, l2], frameon=False, loc='lower left')





l1, =ax2.loglog(Rbins, Sigma_filament0, c='bisque', label='0')
ax2.loglog(Rbins, Sigma_void0, c='bisque', ls='--')
l2,=ax2.loglog(Rbins, Sigma_filament5, c='tomato', label='0.5')
ax2.loglog(Rbins, Sigma_void5, c='tomato', ls='--')
l3, =ax2.loglog(Rbins, Sigma_filament1, c='darkred', label='1')
ax2.loglog(Rbins, Sigma_void1, c='darkred', ls='--')

legend = ax2.legend(handles=[l1, l2, l3], loc='upper right', title='z', frameon=False)
ax2.add_artist(legend)
l1, = ax2.plot([1, 1], [100, 100], c='k', label='filament')
l2, = ax2.plot([1, 1], [100, 100], c='k', ls='--', label='void')
ax2.legend(handles=[l1, l2], frameon=False, loc='lower left')



ax1.set_yticks([])
ax1.set_ylim([5e-3, 1e-1])
ax2.set_ylim([5e-3, 1e-1])


ax1.set_xticks([0.5, 1, 3])
ax1.axes.xaxis.set_ticklabels([0.5, 1, 3])

ax2.set_xticks([0.5, 1, 3])
ax2.axes.xaxis.set_ticklabels([0.5, 1, 3])

ax2.set_ylabel('$\Sigma_s(R)/\overline{\Sigma}_s$')


plt.savefig("figs/9filamentsz.pdf")
plt.tight_layout()
plt.show()